 AOP2 - ESTRUTURA DE DADOS II - ORDENAÇÃO DE DADOS

 Membro 1: Braian Junior Secchin Daros, Matricula: 202413507

In [1]:
# 1. CONFIGURAÇÕES E CONSTANTES GLOBAIS

import os # Biblioteca que permite interação com os arquivos e pastas do computador

M = 3  # Memória interna
F = 3  # Intercalação de 3-Caminhos
NUM_FITAS_TOTAL = 2 * F # 6 fitas auxiliares
NOME_ARQUIVO_ENTRADA = "entrada.txt"
NOME_ARQUIVO_SAIDA = "entrada_ordenda.txt"
PREFIXO_FITA = "fita_temp_"
INFINITY = float('inf')

In [2]:
# 2. ESTRUTURAS DE DADOS OBRIGATÓRIAS
# 1. lista de chaves: (buffer de M=3 posições)
lista_chaves = [None] * M
# 2. lista de fitas: (buffer de M=3 posições)
lista_fitas_origem = [None] * M
# 3. lista de status: (F=3 posições)
lista_status = [True] * F
# 4. fita_corrente:
fita_corrente_saida_idx = 0

In [3]:
# 3. FUNÇÃO DE ORDENAÇÃO OBRIGATÓRIA
def sort_com_fitas(chaves, fitas_origem):
    # Função obrigatória: Ordena a 'lista_chaves' e garante que a 'lista_fitas_origem'
    # seja atualizada para refletir as trocas.
    pares = list(zip(chaves, fitas_origem))
    pares.sort(key=lambda item: item[0] if item[0] is not None else INFINITY)
    for i in range(len(pares)):
        chaves[i] = pares[i][0]
        fitas_origem[i] = pares[i][1]

In [4]:
# 4. FUNÇÕES AUXILIARES
def verificar_arquivo_entrada(nome_arquivo):
    # Verifica se o 'entrada.txt' existe. Se não existir exibe um erro e finaliza o codigo.
    if not os.path.exists(nome_arquivo):
        print(f"ERRO: Arquivo de entrada '{nome_arquivo}' não foi encontrado.")
        print("Por favor, crie o arquivo 'entrada.txt' no mesmo diretório e tente novamente.")
        return False

    print(f"Arquivo '{nome_arquivo}' encontrado. Iniciando processo.")
    return True

def limpar_fitas_temporarias():
    # Apaga todos os arquivos 'fita_temp_*.tmp'
    print("Limpando arquivos temporários...")
    try:
        for i in range(NUM_FITAS_TOTAL):
            nome_fita = f"{PREFIXO_FITA}{i+1}.tmp"
            if os.path.exists(nome_fita):
                os.remove(nome_fita)
        print("Limpeza concluída.")
    except Exception as e:
        print(f"Erro durante a limpeza: {e}")

In [5]:
# 5. FASE 1: GERAÇÃO DOS BLOCOS INICIAIS (DISTRIBUIÇÃO)
def fase_1_distribuicao():
    # Lê 'entrada.txt', ordena blocos de tamanho M=3 e distribui nas fitas de saída iniciais (Fitas 1, 2, 3).
    # Retorna o número total de blocos gerados.
    print(f"--- Iniciando Fase 1: Geração dos Blocos Iniciais (M={M}) ---")
    total_blocos_gerados = 0
    nomes_fitas_saida = [f"{PREFIXO_FITA}{i+1}.tmp" for i in range(F)]
    try:
        with open(NOME_ARQUIVO_ENTRADA, 'r') as f_entrada:
            fitas_saida = [open(nome, 'w') for nome in nomes_fitas_saida]
            fita_saida_atual_idx = 0
            while True: #Loop principal, Cria buffer de leitura
                buffer_leitura = []
                for _ in range(M):
                    linha = f_entrada.readline()
                    if not linha: #Se a linha estiver vazia vai parar o codigo
                        break
                    buffer_leitura.append(int(linha.strip()))
                if not buffer_leitura:#Se a buffer estiver vazio vai parar o codigo
                    break
                buffer_leitura.sort()
                fita_destino = fitas_saida[fita_saida_atual_idx]
                for chave in buffer_leitura:
                    fita_destino.write(f"{chave}\n")
                total_blocos_gerados += 1
                fita_saida_atual_idx = (fita_saida_atual_idx + 1) % F
            for fita in fitas_saida:
                fita.close()
    except FileNotFoundError:#Tratamento de erro caso aconteça de nao encontrar o arquivo no diretorio
        print(f"ERRO: Arquivo de entrada '{NOME_ARQUIVO_ENTRADA}' não encontrado.")
        return 0
    except Exception as e:#Tratamento de erro caso aconteça de o arquivo nao estar no formato correto
        print(f"ERRO na Fase 1: {e}")
        return 0
    print(f"--- Fase 1 Concluída: {total_blocos_gerados} blocos iniciais gerados. ---")
    return total_blocos_gerados


In [6]:
# 6. FASE 2: INTERCALAÇÃO BALANCEADA
def fase_2_intercalacao(num_blocos_total):
    # Executa as passadas de intercalação dos blocos ordenados até que reste apenas um bloco na fita final.
    print(f"\n--- Iniciando Fase 2: Intercalação Balanceada ({F}-Caminhos) ---")
    nomes_fitas = [f"{PREFIXO_FITA}{i+1}.tmp" for i in range(NUM_FITAS_TOTAL)]
    indices_grupo_A = list(range(0, F))     # Fitas [1, 2, 3]
    indices_grupo_B = list(range(F, 2*F))   # Fitas [4, 5, 6]
    indices_fitas_entrada = indices_grupo_A
    indices_fitas_saida = indices_grupo_B
    tamanho_bloco_leitura = M
    passada = 0
    while num_blocos_total > 1:
        passada += 1
        print(f"\n=> Iniciando Passada {passada} de Intercalação")
        print(f"   Lendo blocos de tamanho: {tamanho_bloco_leitura}")
        print(f"   Fitas de Entrada: {[i+1 for i in indices_fitas_entrada]}")
        print(f"   Fitas de Saída: {[i+1 for i in indices_fitas_saida]}")

        # Tenta abrir as fitas de entrada. Se uma fita não existir abre um 'None'
        fitas_entrada = []
        for i in indices_fitas_entrada:
            try:
                fitas_entrada.append(open(nomes_fitas[i], 'r'))
            except FileNotFoundError:
                fitas_entrada.append(None) # Adiciona None se a fita não existir
        fitas_saida = [open(nomes_fitas[i], 'w') for i in indices_fitas_saida]
        global fita_corrente_saida_idx
        fita_corrente_saida_idx = 0
        num_blocos_gerados_nesta_passada = 0
        global lista_status
        lista_status = [True] * F
        # Corrige o status para fitas que não existem
        for i in range(F):
            if fitas_entrada[i] is None:
                lista_status[i] = False
        while any(lista_status):
            num_blocos_gerados_nesta_passada += 1
            fita_destino = fitas_saida[fita_corrente_saida_idx]
            itens_lidos_por_fita = [0] * F
            global lista_chaves, lista_fitas_origem
            lista_chaves = [INFINITY] * F
            lista_fitas_origem = [None] * F
            for i in range(F):
                if lista_status[i]:
                    linha = fitas_entrada[i].readline()
                    if linha:
                        lista_chaves[i] = int(linha.strip())
                        lista_fitas_origem[i] = i
                        itens_lidos_por_fita[i] = 1
                    else:
                        lista_status[i] = False
            while True:
                sort_com_fitas(lista_chaves, lista_fitas_origem)
                chave_menor = lista_chaves[0]
                origem_idx = lista_fitas_origem[0]
                if chave_menor == INFINITY:
                    break
                fita_destino.write(f"{chave_menor}\n")
                if lista_status[origem_idx] and (itens_lidos_por_fita[origem_idx] < tamanho_bloco_leitura):
                    linha = fitas_entrada[origem_idx].readline()
                    if linha:
                        lista_chaves[0] = int(linha.strip())
                        itens_lidos_por_fita[origem_idx] += 1
                    else:
                        lista_status[origem_idx] = False
                        lista_chaves[0] = INFINITY
                        lista_fitas_origem[0] = None
                else:
                    lista_chaves[0] = INFINITY
                    lista_fitas_origem[0] = None
            fita_corrente_saida_idx = (fita_corrente_saida_idx + 1) % F
            if not any(lista_status):
                break
        for f in fitas_entrada:
            if f: f.close() # Fecha apenas se não for None
        for f in fitas_saida:
            f.close()
        num_blocos_total = num_blocos_gerados_nesta_passada
        tamanho_bloco_leitura *= F
        print(f"   Passada {passada} concluída. {num_blocos_total} blocos gerados.")
        indices_fitas_entrada, indices_fitas_saida = indices_fitas_saida, indices_fitas_entrada
    print(f"\n--- Fase 2 Concluída: Intercalação finalizada após {passada} passadas. ---")
    # Encontra o nome da fita final
    nome_fita_final = ""
    for i in indices_fitas_entrada:
        if os.path.exists(nomes_fitas[i]):
             nome_fita_final = nomes_fitas[i]
             break # Pega a primeira fita que existir
    if not nome_fita_final:
        print("ERRO: Nenhuma fita final foi encontrada.")
        return
    try:
        os.rename(nome_fita_final, NOME_ARQUIVO_SAIDA)
        print(f"Arquivo final '{NOME_ARQUIVO_SAIDA}' criado com sucesso.")
    except Exception as e:
        print(f"Erro ao renomear fita final '{nome_fita_final}': {e}")
        try:
            with open(nome_fita_final, 'r') as f_in, open(NOME_ARQUIVO_SAIDA, 'w') as f_out:
                f_out.write(f_in.read())
            print(f"Arquivo final '{NOME_ARQUIVO_SAIDA}' criado por cópia.")
        except Exception as e_copy:
            print(f"Erro fatal ao copiar fita final: {e_copy}")
    limpar_fitas_temporarias()

In [7]:
# 7. FUNÇÃO PRINCIPAL (DRIVER)
def ordenar_arquivo_externo():
    #Função principal que comanda todo o processo de ordenação.

    # 1. Verifica se o arquivo de entrada existe
    if not verificar_arquivo_entrada(NOME_ARQUIVO_ENTRADA):
        return # Para a execução se o arquivo não for encontrado

    # 2. Limpa execuções anteriores
    limpar_fitas_temporarias()
    if os.path.exists(NOME_ARQUIVO_SAIDA):
        os.remove(NOME_ARQUIVO_SAIDA)

    # 3. Fase 1: Gera blocos iniciais ordenados
    num_blocos = fase_1_distribuicao()

    # 4. Fase 2: Intercala os blocos
    if num_blocos > 0:
        fase_2_intercalacao(num_blocos)
    else:
        # Se não houver um arquivo de entrada vazio, cria um arquivo de saída vazio
        open(NOME_ARQUIVO_SAIDA, 'w').close()
        print("Arquivo de entrada estava vazio. Arquivo de saída vazio criado.")

In [10]:
# 8. EXECUÇÃO
ordenar_arquivo_externo()
# Opcional: Verificar o resultado
if os.path.exists(NOME_ARQUIVO_SAIDA):
     print(f"\n--- Verificando as 15 primeiras linhas de '{NOME_ARQUIVO_SAIDA}' ---")
     try:
        with open(NOME_ARQUIVO_SAIDA, 'r') as f_out:
            for i, line in enumerate(f_out):
                if i >= 15: break
                print(f"Linha {i+1}: {line.strip()}")
     except Exception as e:
        print(f"Não foi possível ler o arquivo de saída: {e}")

Arquivo 'entrada.txt' encontrado. Iniciando processo.
Limpando arquivos temporários...
Limpeza concluída.
--- Iniciando Fase 1: Geração dos Blocos Iniciais (M=3) ---
--- Fase 1 Concluída: 167 blocos iniciais gerados. ---

--- Iniciando Fase 2: Intercalação Balanceada (3-Caminhos) ---

=> Iniciando Passada 1 de Intercalação
   Lendo blocos de tamanho: 3
   Fitas de Entrada: [1, 2, 3]
   Fitas de Saída: [4, 5, 6]
   Passada 1 concluída. 57 blocos gerados.

=> Iniciando Passada 2 de Intercalação
   Lendo blocos de tamanho: 9
   Fitas de Entrada: [4, 5, 6]
   Fitas de Saída: [1, 2, 3]
   Passada 2 concluída. 20 blocos gerados.

=> Iniciando Passada 3 de Intercalação
   Lendo blocos de tamanho: 27
   Fitas de Entrada: [1, 2, 3]
   Fitas de Saída: [4, 5, 6]
   Passada 3 concluída. 7 blocos gerados.

=> Iniciando Passada 4 de Intercalação
   Lendo blocos de tamanho: 81
   Fitas de Entrada: [4, 5, 6]
   Fitas de Saída: [1, 2, 3]
   Passada 4 concluída. 3 blocos gerados.

=> Iniciando Passada 5